# Model Context Protocol (MCP) Desteği ile Azure AI Ajanları

Bu not defteri, Model Context Protocol (MCP) araçlarını kullanarak harici MCP sunucularından gelişmiş yetenekler elde edebilen bir zeki ajan oluşturmak için Azure AI Ajanlarını nasıl kullanacağınızı gösterir.


## Gerekli NuGet Paketlerini Yükleme

Öncelikle, Azure AI Agents ile çalışmak için temel işlevselliği sağlayan Azure AI Agents Persistent paketini yüklememiz gerekiyor.


## Anahtarsız Kimlik Doğrulama Faydaları

Bu not defteri, birkaç avantaj sunan **anahtarsız kimlik doğrulamayı** göstermektedir:
- ✅ **Yönetilecek API anahtarı yok** - Azure kimlik tabanlı kimlik doğrulama kullanır
- ✅ **Gelişmiş güvenlik** - Kodda veya yapılandırmada gizli bilgiler saklanmaz
- ✅ **Otomatik kimlik bilgisi yenileme** - Azure kimlik bilgisi yaşam döngüsünü yönetir
- ✅ **Rol tabanlı erişim** - Hassas izinler için Azure RBAC kullanır

`DefaultAzureCredential` otomatik olarak en iyi mevcut kimlik bilgisi kaynağını kullanacaktır:
1. Yönetilen Kimlik (Azure'da çalışırken)
2. Azure CLI kimlik bilgileri (geliştirme sırasında)
3. Visual Studio kimlik bilgileri
4. Ortam değişkenleri (eğer yapılandırılmışsa)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Azure hizmetleriyle kimlik doğrulama için DefaultAzureCredential kullanarak Azure Identity paketini yükleyin.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Gerekli Ad Alanlarını İçe Aktar

Azure AI Agents ve Azure Identity için gerekli ad alanlarını içe aktarın.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Azure AI Agent İstemcisini Yapılandırma (Anahtarsız Kimlik Doğrulama)

Yapılandırma değişkenlerini ayarlayın ve **anahtarsız kimlik doğrulama** kullanarak PersistentAgentsClient oluşturun:
- **projectEndpoint**: Azure AI Foundry proje uç noktası
- **modelDeploymentName**: Dağıtılmış AI modelinin adı (GPT-4.1 nano)
- **mcpServerUrl**: MCP sunucusunun URL'si (Microsoft Learn API)
- **mcpServerLabel**: MCP sunucusunu tanımlamak için bir etiket
- **DefaultAzureCredential**: Yönetilen kimlik, Azure CLI veya diğer kimlik bilgisi kaynaklarını kullanır (API anahtarları gerekmez)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## MCP Araç Tanımı Oluşturma

Microsoft Learn MCP sunucusuna bağlanan bir MCP araç tanımı oluşturun. Bu, ajanın Microsoft Learn içeriğine ve belgelerine erişmesini sağlayacaktır.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## AI Ajanını Oluştur

Belirtilen model ve MCP araçlarıyla kalıcı bir AI ajanı oluşturun. Ajan şu şekilde yapılandırılmıştır:
- GPT-4.1 nano modeli
- Yardım için MCP araçlarını kullanma talimatları
- Microsoft Learn MCP sunucusuna erişim


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Konu Başlığı Oluştur ve Mesaj Gönder

Bir konuşma başlığı oluşturun ve Azure OpenAI ile OpenAI arasındaki farkı sormak için bir kullanıcı mesajı gönderin. Bu, aracının MCP araçlarını kullanarak doğru bilgi sağlama yeteneğini test edecektir.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## MCP Aracı Kaynaklarını Yapılandırma (Anahtarsız)

MCP aracı kaynaklarını ayarlayın. Gerçekten anahtarsız bir yaklaşım için, MCP sunucusu Azure kimlik tabanlı kimlik doğrulamasını destekliyorsa özel başlıkları kaldırabilirsiniz. Aşağıdaki örnek, gerekirse başlıkların nasıl ekleneceğini gösterir, ancak anahtarsız senaryolar için bunlar gerekli olmayabilir.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Aracı Çalıştırmayı Başlat

Kullanıcının mesajını işlemek için bir çalışma oluşturun ve başlatın. Aracı, yanıt oluşturmak için yapılandırılmış MCP araçlarını ve kaynaklarını kullanacaktır.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Çalıştırmayı İzleme ve Araç Onaylarını Yönetme (Anahtarsız)

Aracının çalışma durumunu izleyin ve gerekli araç onaylarını yönetin. Bu döngü:
1. Çalışmanın tamamlanmasını veya bir işlem gerektirmesini bekler
2. Gerekli olduğunda MCP araç çağrılarını otomatik olarak onaylar
3. Anahtarsız kimlik doğrulama için, MCP sunucusu Azure kimliğini destekliyorsa başlıklara gerek olmayabilir


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Konuşma Sonuçlarını Göster

Konudaki tüm mesajları al ve göster, hem kullanıcının sorusunu hem de temsilcinin yanıtını içerecek şekilde. Mesajlar, zaman damgaları ve rol göstergeleriyle kronolojik sırayla görüntülenir.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalardan sorumlu değiliz.
